<a href="https://colab.research.google.com/github/moeedrajpoot1/Project-03-LangChain-Function-Tool-Calling/blob/main/Project03_Function_tool_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install --upgarde langchain-google-genai --quit


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --upgarde


In [28]:
!pip install -U langchain-google-genai

In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key = userdata.get('GEMINI_API_KEY')
)

In [9]:
llm.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-68825262-2c72-4280-8b2d-26a2836eb940-0', usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}})

In [10]:
from langchain_core.tools import tool
import math

@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b

@tool
def square_rt(a: int) -> float:
    """Find square root of a number."""
    return math.sqrt(a)

@tool
def annual_return(a: int) -> float:
    """Find annual return of a number."""
    return (math.sqrt(a) * 85) / 0.15

tools = [add, multiply, square_rt, annual_return]

In [11]:

# Bind tools to the LLM
llm_with_tools = llm.bind_tools(tools)

In [12]:

from IPython.display import display

# invoke an LLM with out tool
ai_msg = llm.invoke("What is annual return of 25")
display(ai_msg)

AIMessage(content='An annual return of 25% means that an investment increased in value by 25% over a one-year period.  For example, if you invested $1000 and it had a 25% annual return, it would be worth $1250 at the end of the year.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3afb0b01-f127-4340-bbcf-f421b3473083-0', usage_metadata={'input_tokens': 9, 'output_tokens': 67, 'total_tokens': 76, 'input_token_details': {'cache_read': 0}})

In [13]:

# invoke an LLM with tools
from langchain_core.messages import HumanMessage, AIMessage
query = "What is annual return of 25"
messages = [HumanMessage(query)]
display(messages)

[HumanMessage(content='What is annual return of 25', additional_kwargs={}, response_metadata={})]

In [14]:

ai_msg_tools = llm_with_tools.invoke(messages)
messages.append(ai_msg_tools)
display(messages)


[HumanMessage(content='What is annual return of 25', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'annual_return', 'arguments': '{"a": 25.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-abaf1516-ca23-4916-9ead-73afb0b935c2-0', tool_calls=[{'name': 'annual_return', 'args': {'a': 25.0}, 'id': '9dc6a66b-66f4-4775-8aab-69ddd5e8f639', 'type': 'tool_call'}], usage_metadata={'input_tokens': 180, 'output_tokens': 4, 'total_tokens': 184, 'input_token_details': {'cache_read': 0}})]

In [15]:
display(ai_msg_tools)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'annual_return', 'arguments': '{"a": 25.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-abaf1516-ca23-4916-9ead-73afb0b935c2-0', tool_calls=[{'name': 'annual_return', 'args': {'a': 25.0}, 'id': '9dc6a66b-66f4-4775-8aab-69ddd5e8f639', 'type': 'tool_call'}], usage_metadata={'input_tokens': 180, 'output_tokens': 4, 'total_tokens': 184, 'input_token_details': {'cache_read': 0}})

In [16]:

print(ai_msg_tools.tool_calls)

[{'name': 'annual_return', 'args': {'a': 25.0}, 'id': '9dc6a66b-66f4-4775-8aab-69ddd5e8f639', 'type': 'tool_call'}]


In [17]:

# Invoke the function / tool
for tool_call in ai_msg_tools.tool_calls:
    selected_tool = {
        "add": add,
        "multiply": multiply,
        "square_rt":square_rt,
        "annual_return":annual_return
      }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    display(tool_msg)
    messages.append(tool_msg)


ToolMessage(content='2833.3333333333335', name='annual_return', tool_call_id='9dc6a66b-66f4-4775-8aab-69ddd5e8f639')

In [18]:
display(messages)


[HumanMessage(content='What is annual return of 25', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'annual_return', 'arguments': '{"a": 25.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-abaf1516-ca23-4916-9ead-73afb0b935c2-0', tool_calls=[{'name': 'annual_return', 'args': {'a': 25.0}, 'id': '9dc6a66b-66f4-4775-8aab-69ddd5e8f639', 'type': 'tool_call'}], usage_metadata={'input_tokens': 180, 'output_tokens': 4, 'total_tokens': 184, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='2833.3333333333335', name='annual_return', tool_call_id='9dc6a66b-66f4-4775-8aab-69ddd5e8f639')]

In [19]:

# Invoke the llm
response = llm_with_tools.invoke(messages)

In [20]:

from IPython.display import Markdown

Markdown(response.content)


Annual return of 25 is 2833.33.